# API服務測試和演示

本筆記本演示如何使用IMDB情感分析API服務：
- 啟動API服務
- 測試單個預測
- 測試批次預測
- 模型切換
- 性能測試

In [ ]:
import requests
import time
import json
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Dict

# API基礎URL
BASE_URL = "http://localhost:8000"

print("API測試工具載入完成！")
print(f"API服務地址: {BASE_URL}")
print("請確保API服務已啟動: python app.py")

In [ ]:
# 健康檢查
def check_health():
    """檢查API服務健康狀態"""
    try:
        response = requests.get(f"{BASE_URL}/health")
        if response.status_code == 200:
            health_data = response.json()
            print("✅ API服務運行正常")
            print(f"狀態: {health_data['status']}")
            print(f"版本: {health_data['version']}")
            print(f"運行時間: {health_data['uptime']:.2f}秒")
            print(f"可用模型: {health_data['available_models']}")
            print(f"當前模型: {health_data['current_model']}")
            return True
        else:
            print(f"❌ API服務異常: {response.status_code}")
            return False
    except requests.exceptions.ConnectionError:
        print("❌ 無法連接到API服務，請確保服務已啟動")
        return False

check_health()

In [ ]:
# 單個預測測試
def test_single_prediction(text: str):
    """測試單個文本預測"""
    data = {"text": text}
    
    try:
        start_time = time.time()
        response = requests.post(f"{BASE_URL}/predict", json=data)
        end_time = time.time()
        
        if response.status_code == 200:
            result = response.json()
            
            print(f"📝 文本: {text[:100]}...")
            print(f"🎯 預測: {result['sentiment']} ({result['confidence']:.3f})")
            print(f"📊 機率: 負面={result['probabilities']['negative']:.3f}, 正面={result['probabilities']['positive']:.3f}")
            print(f"⏱️  推理時間: {result['inference_time']*1000:.1f}ms")
            print(f"🤖 使用模型: {result['model_used']}")
            print(f"🌐 API響應時間: {(end_time-start_time)*1000:.1f}ms")
            
            return result
        else:
            print(f"❌ 預測失敗: {response.status_code} - {response.text}")
            return None
            
    except Exception as e:
        print(f"❌ 請求失敗: {e}")
        return None

# 測試示例
test_cases = [
    "This movie was absolutely fantastic! Great acting and amazing plot.",
    "Terrible movie, waste of time. Poor acting and boring story.",
    "The film was okay, nothing special but not bad either.",
    "One of the best movies I've ever seen! Highly recommended!",
    "Worst movie ever. Don't waste your money on this garbage."
]

print("=" * 60)
print("單個預測測試")
print("=" * 60)

for i, text in enumerate(test_cases, 1):
    print(f"\n測試 {i}:")
    test_single_prediction(text)

In [ ]:
# 批次預測測試
def test_batch_prediction(texts: List[str]):
    """測試批次預測"""
    data = {"texts": texts}
    
    try:
        start_time = time.time()
        response = requests.post(f"{BASE_URL}/predict/batch", json=data)
        end_time = time.time()
        
        if response.status_code == 200:
            result = response.json()
            
            print(f"📊 批次大小: {result['total_texts']}")
            print(f"⏱️  總時間: {result['total_time']*1000:.1f}ms")
            print(f"📈 平均時間: {result['average_time']*1000:.1f}ms/個")
            print(f"🌐 API響應時間: {(end_time-start_time)*1000:.1f}ms")
            
            print("\n詳細結果:")
            for i, res in enumerate(result['results'], 1):
                sentiment_emoji = "😊" if res['sentiment'] == 'positive' else "😞"
                print(f"{i}. {sentiment_emoji} {res['sentiment']} ({res['confidence']:.3f}) - {res['text'][:50]}...")
            
            return result
        else:
            print(f"❌ 批次預測失敗: {response.status_code} - {response.text}")
            return None
            
    except Exception as e:
        print(f"❌ 請求失敗: {e}")
        return None

print("\n" + "=" * 60)
print("批次預測測試")
print("=" * 60)

batch_result = test_batch_prediction(test_cases)

In [ ]:
# 獲取可用模型
def get_available_models():
    """獲取可用模型列表"""
    try:
        response = requests.get(f"{BASE_URL}/models")
        if response.status_code == 200:
            models = response.json()
            print("📋 可用模型列表:")
            for model in models:
                status = "✅ 已載入" if model['is_loaded'] else "⏸️ 未載入"
                print(f"  - {model['name']} ({model['type']}) {status}")
            return models
        else:
            print(f"❌ 獲取模型列表失敗: {response.status_code}")
            return []
    except Exception as e:
        print(f"❌ 請求失敗: {e}")
        return []

available_models = get_available_models()

In [ ]:
# 性能測試
def performance_test(num_requests: int = 10):
    """API性能測試"""
    test_text = "This is a great movie with excellent acting and plot!"
    times = []
    
    print(f"🚀 開始性能測試 ({num_requests} 次請求)...")
    
    for i in range(num_requests):
        start_time = time.time()
        response = requests.post(f"{BASE_URL}/predict", json={"text": test_text})
        end_time = time.time()
        
        if response.status_code == 200:
            response_time = (end_time - start_time) * 1000
            times.append(response_time)
            if (i + 1) % 5 == 0:
                print(f"已完成 {i + 1}/{num_requests} 次請求")
        else:
            print(f"請求 {i + 1} 失敗")
    
    if times:
        avg_time = sum(times) / len(times)
        min_time = min(times)
        max_time = max(times)
        
        print(f"\n📊 性能測試結果:")
        print(f"平均響應時間: {avg_time:.1f}ms")
        print(f"最快響應時間: {min_time:.1f}ms")
        print(f"最慢響應時間: {max_time:.1f}ms")
        print(f"吞吐量: {1000/avg_time:.1f} 請求/秒")
        
        # 繪製響應時間分佈
        plt.figure(figsize=(10, 6))
        plt.hist(times, bins=20, alpha=0.7, color='skyblue', edgecolor='black')
        plt.axvline(avg_time, color='red', linestyle='--', label=f'平均: {avg_time:.1f}ms')
        plt.xlabel('響應時間 (ms)')
        plt.ylabel('頻率')
        plt.title('API響應時間分佈')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()
        
        return times
    else:
        print("❌ 性能測試失敗")
        return []

# 執行性能測試
performance_times = performance_test(20)

In [ ]:
# 互動式測試
def interactive_test():
    """互動式情感分析測試"""
    print("🎬 互動式IMDB情感分析測試")
    print("輸入 'quit' 結束測試\n")
    
    while True:
        try:
            text = input("請輸入電影評論: ").strip()
            
            if text.lower() == 'quit':
                print("測試結束！")
                break
            
            if not text:
                print("請輸入有效的文本")
                continue
            
            result = test_single_prediction(text)
            print("-" * 50)
            
        except KeyboardInterrupt:
            print("\n測試中斷！")
            break
        except Exception as e:
            print(f"發生錯誤: {e}")

# 取消註釋下面這行來運行互動式測試
# interactive_test()

In [ ]:
print("\n🎉 API測試完成！")
print("\n📖 更多功能:")
print(f"- Web介面: {BASE_URL}/")
print(f"- API文檔: {BASE_URL}/docs")
print(f"- ReDoc文檔: {BASE_URL}/redoc")
print(f"- 健康檢查: {BASE_URL}/health")
print(f"- 模型列表: {BASE_URL}/models")